In [1]:
import EOVoxelCraft as eovc
import geopandas as gpd
from pathlib import Path

c:\Users\hoeh_pa\AppData\Local\miniconda3\envs\sits\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
crafter = eovc.init('cdse')
credentials_path = Path(r"C:\Users\hoeh_pa\Desktop\EOVoxelCraft\credentials\cdse_credentials.json")
crafter.set_credentials(credentials_path)

In [3]:
collections = crafter.retrieve_collections()
print(collections)

['COP-DEM', 'S2GLC', 'TERRAAQUA', 'SENTINEL-3', 'SENTINEL-5P', 'SENTINEL-1-RTC', 'SENTINEL-1', 'SMOS', 'LANDSAT-7', 'CCM', 'LANDSAT-5', 'LANDSAT-8', 'ENVISAT', 'SENTINEL-6', 'GLOBAL-MOSAICS', 'SENTINEL-2']


In [4]:
arguments = dict(
    shp=gpd.read_file(r"C:\Users\hoeh_pa\Desktop\EOVoxelCraft\demo_files\data\TUM_OTN.shp.zip"), 
    resolution=20,
    collection='SENTINEL-2', 
    start_date='2021-06-01', 
    end_date='2021-07-15', 
    download_folder=r'C:\Users\hoeh_pa\Desktop\EOVoxelCraft\demo_files\data\downloads', 
    num_workers=8
    )

In [5]:
items = crafter.search(**arguments)

In [6]:
unique_items = list({item.id: item for item in items}.values())

In [9]:
selected_items = []
for item in unique_items:
    if item.properties["tileId"] == "32UQU" and item.properties['processingLevel'] == "S2MSI2A" and item.properties['cloudCover'] < 10.0:
        selected_items.append(item)
print(f"Selected items: {len(selected_items)}")
selected_items

Selected items: 3


[<Item id=S2B_MSIL2A_20210617T100559_N0300_R022_T32UQU_20210617T131219.SAFE>,
 <Item id=S2B_MSIL2A_20210710T101559_N0301_R065_T32UQU_20210710T132321.SAFE>,
 <Item id=S2B_MSIL2A_20210617T100559_N0500_R022_T32UQU_20230321T221728.SAFE>]

In [8]:
# cube = crafter.download(selected_items, create_minicube=True, delete_zip=True)
output_dir = r"C:\Users\hoeh_pa\Desktop\EOVoxelCraft\demo_files\data\downloads\S2"
cube = crafter.build_minicube(output_dir)